In [1]:
# on server: 'screen' ,then start script
# use 'strg+a d' to return to terminal
# use 'screen -r' to return to screen

import numpy as np
import json
import os

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.layers import Input,Dense,LSTM,GRU
from keras.layers import Bidirectional
from keras.utils import plot_model
# os.environ['CUDA_VISIBLE_DEVICES']='0'

Using TensorFlow backend.


In [2]:
import rnn_functions as rf

In [3]:
path='models/baseline_biderec'
# rnn parameters
hidden_size = 100 #100 is the standard
batch_size = 512 #for the training on the GPU this to be has to very large, otherwise the GPU is used very inefficiently
epochs = 100

#glove embedding parameters
glove_dir = '../glove/glove.6B.100d.txt'
embedding_dim = 100
eval_split = 0.2 

In [4]:
#open SQuAD-dataset and extract the relevant data from the json-file
#to a easier readable/accessible dictionary
with open('SQuAD/train-v2.0.json') as file:
    train=json.load(file)
train_context=[]
train_question=[]
train_answer=[]
train_new={'context':train_context,'question':train_question,'answer':train_answer}
for j,data in enumerate(train['data']):
    for i,paragraph in enumerate(data['paragraphs']):
        context=paragraph['context']
        for qas in paragraph['qas']:
            #create a dataset with only the answerable questions
            if (qas['is_impossible']==False):
            #FIXME add 'bos' and 'eos' to data
                train_new['context'].append(context)
                train_new['question'].append(qas['question'])
                train_new['answer'].append(qas['answers'][0]['text'])

In [5]:
#concatenate the data in one vector for preprocessing
train_all=[]
for line in train_new['context']:
    train_all.append(line)
for line in train_new['question']:
    train_all.append(line)
for line in train_new['answer']:
    train_all.append(line)
print('len train_all: ',len(train_all))

len train_all:  260463


In [29]:
vocab = set()
vocab |= set(text_to_word_sequence('eos'))
vocab |= set(text_to_word_sequence('bos'))
for text in train_all:
    vocab |= set(text_to_word_sequence(text))
vocab = sorted(vocab)
vocab_size = len(vocab) + 1
print('vocab shape: ',vocab_size)

vocab shape:  91927


In [31]:
#prepare the data to use as input of the rnn
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_all)
context_sequences = tokenizer.texts_to_sequences(train_new['context'])
question_sequences = tokenizer.texts_to_sequences(train_new['question'])
answer_sequences = tokenizer.texts_to_sequences(train_new['answer'])

max_len_context=max(map(len,context_sequences))
max_len_question=max(map(len,question_sequences))
max_len_answer=max(map(len,answer_sequences))

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# context = pad_sequences(context_sequences, maxlen=max_len_context)
# question = pad_sequences(question_sequences, maxlen=max_len_question)
# answer = pad_sequences(answer_sequences, maxlen=max_len_answer)

context=context_sequences
question=question_sequences
answer=answer_sequences

context_len=len(context)
question_len=len(question)
answer_len=len(answer)

print('Shape of context tensor:', context_len)
print('Shape of question tensor:', question_len)
print('Shape of answer tensor:', answer_len)

Found 91925 unique tokens.
Shape of context tensor: 86821
Shape of question tensor: 86821
Shape of answer tensor: 86821


In [17]:
# # split the data into a training set and an evaluation set
# indices = np.arange(context.shape[0])
# np.random.shuffle(indices)
# context = context[indices]
# question = question[indices]
# answer = answer[indices]
# num_eval_samples = int(eval_split * context.shape[0])

# x_train_context = context[:-num_eval_samples]
# x_train_question = question[:-num_eval_samples]
# y_train_answer = answer[:-num_eval_samples]

# x_eval_context = context[-num_eval_samples:]
# x_eval_question = question[-num_eval_samples:]
# y_eval_answer = answer[-num_eval_samples:]

In [18]:
#FIX_ME: add glove download
# https://nlp.stanford.edu/projects/glove/
#get glove embeddings
embeddings_index = {}
f = open(glove_dir)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [19]:
#extract the glove-embedding to a matrix
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
# # create non-trainable embedding layers
# # for the context and the question each
# context_embedding_layer = Embedding(len(word_index) + 1,
#                             embedding_dim,
#                             weights=[embedding_matrix],
#                             input_length=max_len_context,
#                             trainable=False,
#                             name='context_embedding')
# question_embedding_layer = Embedding(len(word_index) + 1,
#                             embedding_dim,
#                             weights=[embedding_matrix],
#                             input_length=max_len_question,
#                             trainable=False,
#                             name='question_embedding')

In [39]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
x = Embedding(vocab_size, embedding_dim,trainable=False)(encoder_inputs)
x, state_h, state_c = LSTM(latent_dim,
                           return_state=True)(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
x = Embedding(vocab_size, embedding_dim,trainable=False)(decoder_inputs)
x = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)
decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!

# model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.2)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, None, 100)    9192700     input_13[0][0]                   
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, None, 100)    9192700     input_14[0][0]                   
__________________________________________________________________________________________________
lstm_13 (L

In [38]:
# print('Build model...')

# # encoder input data -> context+question
# context_layer = layers.Input(shape=(None,context_len), dtype='int32',name='Context_input')
# embedded_context = context_embedding_layer(context_layer)
# lstm_context = LSTM(hidden_size,return_state=True)
# encoder_context,state_context_h,state_context_c = lstm_context(embedded_context)
# # context_states = [state_context_h,state_context_c]

# # question_layer = layers.Input(shape=(max_len_question,), dtype='int32',name='Question_input')
# # embedded_question = question_embedding_layer(question_layer)
# # lstm_question = LSTM(hidden_size,return_state=True)
# # encoder_question,state_question_h,state_question_c= lstm_question(embedded_question)
# # # question_states = [state_question_h,state_question_c]

# encoder_states=[state_context_h,state_context_c]#,state_question_h,state_question_c]

# # encoder_inputs_merged = layers.concatenate([encoder_context,encoder_question])

# #decoder input data -> answer
# decoder_inputs= layers.Input(shape=(None,answer_len))
# decoder_lstm = LSTM(hidden_size,return_sequences=True,return_state=True)
# decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)

# decoder_dense = Dense(answer_len,activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)

# model=Model([context_layer,decoder_inputs],decoder_outputs)

# # preds = layers.Dense(max_len_answer, activation='softmax',name='Output_layer')(encoder_inputs_merged)

# # model = Model([context_layer, question_layer], preds)

# ###################
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy', #sparse cater ,,reshape batch_size,time_step,vocab_size 
#               metrics=['accuracy'])                           #-> batch_size*time_step,vocab_size
# print('successfully built the model')
# print(model.summary())

In [10]:
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [11]:
# print('Training')
# #flatten y_train
# model.fit([x_train_context, x_train_question], y_train_answer,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.05)

# print('Evaluation')
# loss, acc = model.evaluate([x_eval_context, x_eval_question], y_eval_answer,
#                            batch_size=batch_size)
# print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

In [12]:
# print('save model')
# # plot_model(model, to_file=path+'/model.png')
# model.save_weights('models/baseline/baseline_model.h5') #save weights
# model_json = model.to_json()
# with open("models/baseline/baseline_model.json",'w') as json_file:
#     json_file.write(model_json)

In [13]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [14]:
print(np.shape(input_characters))

(69,)


In [15]:
input_characters[2]

'$'

In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.